# gathering thingiverse things

an effective scraping and caching strategy for gathering information on thingiverse.

each activity requires:
1. making requests
2. tidying requests
3. creating tasks to save salient data, create a file witht he missing columsn


1. gather the index of things from search
2. save the index to disk as parquet
3. read the index of things from disks
4. save the things to disc

the second request is solely to get the description information about the thing cause that is not included in the search response
despite the openapi schema saying that it should be


In [2]:
    import os, pandas, time, random, platformdirs, requests_cache
    from toolz.curried import *
    from pandas import DataFrame, Series, Index
    __import__("dotenv").load_dotenv()
    params = dict(access_token=os.environ["THINGIVERSE_ACCESS_TOKEN"])
    cache = platformdirs.user_cache_path("a11yhood") / "thingiverse"
    search_session = requests_cache.CachedSession(cache / "search_responses.sqlite")
    thing_session = requests_cache.CachedSession(cache / "thing_responses.sqlite")

In [18]:
    def search_tags(term="assistive+technology", pages=None, per_page=30):
        """search thing"""
        page = 1
        while not pages or page <= pages:
            response = search_session.get(f"https://api.thingiverse.com/search/{term}", params | dict(per_page=per_page, page=page, type="thing"),)
            if pages is None:
                pages = (response.json()["total"] // per_page) + 1
            page += 1

In [19]:
    search_tags()

In [20]:
    results = (
        Series(search_session.cache.urls())
        .apply(search_session.get)
        .apply(operator.methodcaller("json"))
        .apply(lambda x: x.get("hits", []))
        .explode()
        .apply(Series)
        .set_index("public_url")
    )

In [24]:
    for j, (i, row) in enumerate(results.dropna(subset="id").iterrows()):
        time.sleep(random.random()*2)
        thing_session.get(row.url, params=params)
        print(i, j)

https://www.thingiverse.com/thing:1593407 0
https://www.thingiverse.com/thing:140519 1
https://www.thingiverse.com/thing:2596644 2
https://www.thingiverse.com/thing:121869 3
https://www.thingiverse.com/thing:3156378 4
https://www.thingiverse.com/thing:1102579 5
https://www.thingiverse.com/thing:3717730 6
https://www.thingiverse.com/thing:1104281 7
https://www.thingiverse.com/thing:3093824 8
https://www.thingiverse.com/thing:1720562 9
https://www.thingiverse.com/thing:1499148 10
https://www.thingiverse.com/thing:1055457 11
https://www.thingiverse.com/thing:49936 12
https://www.thingiverse.com/thing:2823963 13
https://www.thingiverse.com/thing:3635274 14
https://www.thingiverse.com/thing:2930181 15
https://www.thingiverse.com/thing:2577404 16
https://www.thingiverse.com/thing:1089803 17
https://www.thingiverse.com/thing:2960433 18
https://www.thingiverse.com/thing:2257912 19
https://www.thingiverse.com/thing:2768601 20
https://www.thingiverse.com/thing:2863119 21
https://www.thingiverse.

In [33]:
(
    Series(thing_session.cache.responses)
    .apply(operator.methodcaller("json"))
    .apply(Series)
)

,id,name,thumbnail,url,public_url,creator,added,modified,is_published,is_wip,...,root_comment_count,moderation,is_derivative,ancestors,type_name,is_banned,is_comments_disabled,needs_moderation,is_decoy,zip_data
00fdfcd14237dd21,1098836,e-NABLE Phone Home Prototype 3,https://cdn.thingiverse.com/renders/00/a0/a6/2...,https://api.thingiverse.com/things/1098836,https://www.thingiverse.com/thing:1098836,"{'id': 173593, 'name': 'deltamodulator', 'firs...",2015-10-29T06:48:49+00:00,2015-10-29T06:48:49+00:00,1,1,...,0,,True,"[{'id': 596966, 'name': 'Raptor Reloaded by e-...",Thing,False,False,0,0,{'files': [{'name': 'e-NABLE_Phone_Home_10.ino...
017ac16bf4936a98,2881400,3D Printed Arm Socket,https://cdn.thingiverse.com/renders/73/6e/58/d...,https://api.thingiverse.com/things/2881400,https://www.thingiverse.com/thing:2881400,"{'id': 184877, 'name': 'sujana', 'first_name':...",2018-04-27T04:06:29+00:00,2018-04-27T04:06:29+00:00,1,0,...,0,,False,[],Thing,False,False,0,0,{'files': [{'name': 'Arm_Socket_C_27042018.stl...
0182e0eb0261935c,2061214,VESA Mount for Overhead Camera (IPevo HD Plus),https://cdn.thingiverse.com/renders/e2/9b/89/2...,https://api.thingiverse.com/things/2061214,https://www.thingiverse.com/thing:2061214,"{'id': 1276407, 'name': 'atmakers', 'first_nam...",2017-01-25T05:09:19+00:00,2017-01-25T05:09:19+00:00,1,0,...,0,,False,[],Thing,False,False,0,0,{'files': [{'name': 'VESA_Mount_Overhead_Camer...
01de48b8131edf9c,3710570,Interatomic Bonds Tactile Model,https://cdn.thingiverse.com/renders/df/ba/57/9...,https://api.thingiverse.com/things/3710570,https://www.thingiverse.com/thing:3710570,"{'id': 2652477, 'name': 'ntaufscar', 'first_na...",2019-06-24T12:12:19+00:00,2019-06-24T12:12:19+00:00,1,1,...,0,,False,[],Thing,False,False,0,0,"{'files': [{'name': 'ElectronExchange.stl', 'u..."
0261cb5c628ba03f,6586072,Dolphin Flexi Fidget Toy - print in place,https://cdn.thingiverse.com/assets/43/bb/e6/78...,https://api.thingiverse.com/things/6586072,https://www.thingiverse.com/thing:6586072,"{'id': 5595608, 'name': 'timmytombstone1', 'fi...",2024-04-19T17:30:42+00:00,2024-04-19T17:30:42+00:00,1,0,...,1,,False,[],Thing,False,False,0,0,"{'files': [{'name': 'dolphin.stl', 'url': 'htt..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
facf2d0592054807,359366,Accoppia Calze - Strumento di tecnologia assis...,https://cdn.thingiverse.com/renders/ae/99/77/7...,https://api.thingiverse.com/things/359366,https://www.thingiverse.com/thing:359366,"{'id': 241958, 'name': 'PhyCo', 'first_name': ...",2014-06-10T18:03:05+00:00,2014-06-10T18:03:05+00:00,1,0,...,0,,False,[],Thing,False,False,0,0,"{'files': [{'name': 'accoppiacalze2.STL', 'url..."
fbc705e5998a6f4d,2969609,OT student project : hand guide for lock / gui...,https://cdn.thingiverse.com/renders/bb/04/02/d...,https://api.thingiverse.com/things/2969609,https://www.thingiverse.com/thing:2969609,"{'id': 419272, 'name': 'Pole_ergo', 'first_nam...",2018-06-20T08:09:31+00:00,2018-06-20T08:09:31+00:00,1,0,...,0,,False,[],Thing,False,False,0,0,"{'files': [{'name': 'serrure.stl', 'url': 'htt..."
fc434d2e733bd132,683496,Modified RIT Arm Parts,https://cdn.thingiverse.com/renders/33/44/e5/f...,https://api.thingiverse.com/things/683496,https://www.thingiverse.com/thing:683496,"{'id': 248357, 'name': 'masnart39', 'first_nam...",2015-02-15T13:01:20+00:00,2015-02-15T13:01:20+00:00,1,1,...,8,,True,"[{'id': 604380, 'name': 'Modified RIT Arm Part...",Thing,False,False,0,0,{'files': [{'name': 'forearm-cup-modified.STL'...
fc7f60b1f323f996,2513492,I8-module_open°lights,https://cdn.thingiverse.com/renders/ac/5a/41/7...,https://api.thingiverse.com/things/2513492,https://www.thingiverse.com/thing:2513492,"{'id': 126601, 'name': 'FabLab-Berlin', 'first...",2017-09-02T17:33:01+00:00,2017-09-02T17:33:01+00:00,1,0,...,1,,False,[],Thing,False,False,0,0,{'files': [{'name': 'Circuit_Diagram_Module_I....
